**Reference : [Towards_de_anonymizing_the_data](https://www.kaggle.com/raddar/towards-de-anonymizing-the-data-some-insights)**<br/>
<br/>
**Fixed by LazyrIJ**


# This is going to be epic... sit back, relax and enjoy!

<br/>
이전의 노트북에서 target은 log 변환 되었으며, 2**target을 통하여 복원할 수 있었음을 밝혔다. 나의 진짜 목표는 product_sum의 비율 추측이었다.<br/>
<br/>

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', '{:.10f}'.format)

train = pd.read_csv('../input/train.csv')
historical_transactions = pd.read_csv('../input/historical_transactions.csv')
new_merchant_transactions = pd.read_csv('../input/new_merchant_transactions.csv')

<br/>
이전의 커널과 같이 데이터를 변환한다.<br/>
<br/>

In [12]:
new_merchant_transactions['purchase_amount_new'] = np.round(new_merchant_transactions['purchase_amount'] / 0.00150265118 + 497.06,2)
historical_transactions['purchase_amount_new'] = np.round(historical_transactions['purchase_amount'] / 0.00150265118 + 497.06,2)
train['target_raw'] = 2**train['target']

<br/>
이제 그 비율이 의미가 있다는 것을 증명하기 위해서, new_merchant_transaction_table에서 거래가 없는 merchant_id가 1개 뿐인 card_id와 같은 간단한 사례를 찾았다.<br/>
몇 가지 작업으로 나는 merchant_id가 M_과 같은 (merchant_id == ㅡ_ID_fc7d7969c3) 분석하기에 완벽한 값을 찾았다.<br/>
<br/>

In [3]:
#card_id에 따른 merchant_id의 유니크 개수
merchant_counts = historical_transactions\
.groupby(['card_id'])['merchant_id'].nunique()\
.reset_index(name = 'merchant_n')
#merchant_id의 유니크 개수가 1개 뿐인 데이터
one_merchant = merchant_counts[merchant_counts['merchant_n']==1].reset_index(drop=True)
#다시 원래 데이터에서, merchant_id의 유니크 개수가 1인 card_id만 가져옴
dat = historical_transactions.loc[historical_transactions['card_id'].isin(one_merchant['card_id'])]
#new_merchant_transactions에는 없는 card_id만 가져옴
dat = dat.loc[~dat['card_id'].isin(new_merchant_transactions['card_id'])]
#그 중 merchant_id 가 M_ID_fc7d796c3인 것만 가져옴
dat = dat.loc[dat.merchant_id=='M_ID_fc7d7969c3'].reset_index(drop=True)

In [8]:
dat.head(3)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_ba26c0f556,-1,Y,1,B,661,M_ID_fc7d7969c3,-1,-0.7170050400,2018-01-21 10:50:35,nan,-1,8
1,Y,C_ID_ba26c0f556,-1,Y,1,B,661,M_ID_fc7d7969c3,-3,-0.7170050400,2017-11-21 10:49:52,nan,-1,8
2,Y,C_ID_ba26c0f556,-1,Y,1,B,661,M_ID_fc7d7969c3,0,-0.7170050400,2018-02-21 08:43:04,nan,-1,8


<br/>
해당 card_id에 대하여 살펴본다.<br/>
먼저 위 dat 데이터프레임에서 card_id별로 merchard_id의 유니크 값을 보면 전부 1개 뿐이다.<br/>
<br/>

In [10]:
for card in dat['card_id'].head(3):
    unique_merchant = historical_transactions[historical_transactions.card_id=='C_ID_ba26c0f556']\
                      ['merchant_id'].unique()
    print('[*]{} --> {}'.format(card, unique_merchant))

[*]C_ID_ba26c0f556 --> ['M_ID_fc7d7969c3']
[*]C_ID_ba26c0f556 --> ['M_ID_fc7d7969c3']
[*]C_ID_ba26c0f556 --> ['M_ID_fc7d7969c3']


In [4]:
historical_transactions[historical_transactions.card_id=='C_ID_b3c7ff9e19'].sort_values('purchase_date')

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
27771438,Y,C_ID_b3c7ff9e19,-1,Y,1,B,661,M_ID_fc7d7969c3,-13,-0.7019785300,2017-01-03 15:28:20,nan,-1,8
27771445,Y,C_ID_b3c7ff9e19,-1,Y,1,B,661,M_ID_fc7d7969c3,-12,-0.7019785300,2017-02-03 14:57:17,nan,-1,8
27771441,Y,C_ID_b3c7ff9e19,-1,Y,1,B,661,M_ID_fc7d7969c3,-11,-0.7019785300,2017-03-03 18:27:05,nan,-1,8
27771447,Y,C_ID_b3c7ff9e19,-1,Y,1,B,661,M_ID_fc7d7969c3,-10,-0.7019785300,2017-04-03 13:05:07,nan,-1,8
27771449,Y,C_ID_b3c7ff9e19,-1,Y,1,B,661,M_ID_fc7d7969c3,-9,-0.7019785300,2017-05-04 11:08:14,nan,-1,8
27771437,Y,C_ID_b3c7ff9e19,-1,Y,1,B,661,M_ID_fc7d7969c3,-8,-0.7019785300,2017-06-03 13:57:01,nan,-1,8
27771444,Y,C_ID_b3c7ff9e19,-1,Y,1,B,661,M_ID_fc7d7969c3,-7,-0.7019785300,2017-07-03 15:01:43,nan,-1,8
27771440,Y,C_ID_b3c7ff9e19,-1,Y,1,B,661,M_ID_fc7d7969c3,-6,-0.6899573200,2017-08-03 16:22:15,nan,-1,8
27771446,Y,C_ID_b3c7ff9e19,-1,Y,1,B,661,M_ID_fc7d7969c3,-5,-0.6899573200,2017-09-03 15:59:53,nan,-1,8
27771448,Y,C_ID_b3c7ff9e19,-1,Y,1,B,661,M_ID_fc7d7969c3,-3,-0.6899573200,2017-11-03 17:37:09,nan,-1,8


In [5]:
#위 데이터의 card_id중 하나를 선택하여 new_merchant_transactions에서 검색
new_merchant_transactions[new_merchant_transactions.card_id=='C_ID_b3c7ff9e19'].sort_values('purchase_date')

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id


<br/>
결제는 대략 같은 달에 발생하였으며, purchase_amount도 29.9 -> 37.9로 증가했다.<br/>
<br/>

<br/>
merchant_id = M_ 은 왜 특별할까. 인터넷 결제 가능성이 가장 높은 것으로 나타났다. 또한 purchase_amount는 뚜렷한 몇 가지 범주로 나누어진다.<br/>
<br/>

In [13]:
#merchant_id 가 M_ID_fc7d7969c3인 purchase_amount_new의 count
historical_transactions.loc[historical_transactions.merchant_id=='M_ID_fc7d7969c3'].groupby('purchase_amount_new')['card_id'].count()

purchase_amount_new
1.0000000000     45054
9.5000000000         2
13.9500000000        1
19.9000000000    19709
22.9000000000    31680
27.9000000000    33017
29.9000000000    24097
37.9000000000    23480
Name: card_id, dtype: int64

<br/>
위 정보는 target이 어떠한 비율이라는 가설을 위해 매우 유용한 정보이며, purchase_amount 비율은 합리적인 후보처럼 보인다. (좋은 feature?)<br/>
<br/>

<br/>
여기서 우리가 아는 것들을 정리해보면<br/>

- merchant가 구독 기반의 상품을 판매
- card_id는 매월 자동 결제
- 구독 비용은 변경 가능(대부분 상승)
- 19.10, 22.90, 27.90, 29.90, 37.90 총 5종류의 상품 (1은 결제 활성화 비용인 것 같다)

<br/>

<br/>
이제 target과 상관관계를 살펴보자.<br/>
<br/>

In [14]:
dat = dat.merge(train, on = 'card_id')

In [15]:
dat.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,first_active_month,feature_1,feature_2,feature_3,target,target_raw
0,Y,C_ID_ba26c0f556,-1,Y,1,B,661,M_ID_fc7d7969c3,-1,-0.7170050400,2018-01-21 10:50:35,nan,-1,8,2017-08,2,2,0,0.0000000000,1.0000000000
1,Y,C_ID_ba26c0f556,-1,Y,1,B,661,M_ID_fc7d7969c3,-3,-0.7170050400,2017-11-21 10:49:52,nan,-1,8,2017-08,2,2,0,0.0000000000,1.0000000000
2,Y,C_ID_ba26c0f556,-1,Y,1,B,661,M_ID_fc7d7969c3,0,-0.7170050400,2018-02-21 08:43:04,nan,-1,8,2017-08,2,2,0,0.0000000000,1.0000000000
3,Y,C_ID_ba26c0f556,-1,Y,1,B,661,M_ID_fc7d7969c3,-2,-0.7170050400,2017-12-21 08:44:22,nan,-1,8,2017-08,2,2,0,0.0000000000,1.0000000000
4,Y,C_ID_ba26c0f556,-1,Y,1,B,661,M_ID_fc7d7969c3,-5,-0.7170050400,2017-09-21 11:22:47,nan,-1,8,2017-08,2,2,0,0.0000000000,1.0000000000


In [17]:
dat.groupby('target_raw')['card_id'].count().reset_index(name='n').head(10)

,target_raw,n
0,0.0000000001,23
1,0.0218079291,13
2,0.5000000000,32
3,0.6666666670,2
4,0.7132616494,6
5,0.7991967880,5
6,0.9998680889,3
7,1.0000000000,2294
8,1.0001319285,17
9,1.0001792085,15


<br/>
card_id의 대부분은 target_raw = 1에 집중되어있다. 특별한 규칙을 찾고 클러스터링이 가능하기 때문에 이는 좋은 소식이다.<br/>
target_raw = 1인 card_id를 확인해보자.<br/>
<br/>

In [20]:
dat.loc[dat['target_raw']==1,
        ['card_id','purchase_date',
         'purchase_amount_new',
         'target_raw']]\
.sort_values(['card_id','purchase_date']).head(10)

,card_id,purchase_date,purchase_amount_new,target_raw
1282,C_ID_01857b7ee5,2017-01-01 12:57:30,nan,1.0000000000
1292,C_ID_01857b7ee5,2017-01-01 15:07:07,nan,1.0000000000
1288,C_ID_01857b7ee5,2017-01-01 15:07:11,nan,1.0000000000
1285,C_ID_01857b7ee5,2017-02-01 10:17:09,nan,1.0000000000
1296,C_ID_01857b7ee5,2017-03-01 11:05:33,nan,1.0000000000
1291,C_ID_01857b7ee5,2017-04-01 06:57:57,nan,1.0000000000
1299,C_ID_01857b7ee5,2017-05-01 08:28:14,nan,1.0000000000
1290,C_ID_01857b7ee5,2017-06-01 07:16:42,nan,1.0000000000
1294,C_ID_01857b7ee5,2017-07-01 07:16:06,nan,1.0000000000
1283,C_ID_01857b7ee5,2017-08-01 09:16:37,nan,1.0000000000


<br/>
purchase_amount_new에 일부 차이가 존재하며, target_raw가 왜 1인지 아직 알 수 없다.<br/>
<br/>

<br/>
purchase_amount_new와 관계가 깊은 것은 target_raw가 아니라 다른 값들인데, 먼저 purchase_amount_new를 살펴보고, 이 값들을 기준으로 어떠한 가능한 비율을 계산해보자.<br/>
<br/>

In [22]:
#merchant_id=='M_ID_fc7d7969c3'에서 나타났던 purchase_amount_new
prices = [19.90, 22.90, 27.90, 29.90, 37.90]
prices_ratios = sorted({ i/j for j in prices for i in prices})
prices_ratios

[0.525065963060686,
 0.604221635883905,
 0.6655518394648829,
 0.7132616487455197,
 0.7361477572559366,
 0.765886287625418,
 0.7889182058047494,
 0.8207885304659498,
 0.868995633187773,
 0.9331103678929766,
 1.0,
 1.07168458781362,
 1.150753768844221,
 1.2183406113537119,
 1.2675585284280937,
 1.3056768558951966,
 1.3584229390681004,
 1.4020100502512562,
 1.5025125628140703,
 1.6550218340611353,
 1.9045226130653268]

<br/>
이 숫자들은 큰 의미가 없다.<br/>
하지만 merchant_id의 특정 값에서 나왔던 purchase_amount와<br/>
target_raw의 고유값을 비교해보면 중복되는 값이 있음을 알 수 있다.<br/>
<br/>

In [33]:
#target_raw unique
unique_target_raw = dat.groupby('target_raw').values
unique_target_raw

array([9.99999999e-11, 2.18079291e-02, 5.00000000e-01, 6.66666667e-01,
       7.13261649e-01, 7.99196788e-01, 9.99868089e-01, 1.00000000e+00,
       1.00013193e+00, 1.00017921e+00, 1.09410480e+00, 1.09842520e+00,
       1.10917031e+00, 1.11799410e+00, 1.13192612e+00, 1.13377927e+00,
       1.15197568e+00, 1.17921147e+00, 1.21834061e+00, 1.24671052e+00,
       1.35842294e+00, 1.44076517e+00, 2.00000000e+00, 2.03584230e+00,
       2.07526882e+00, 2.43668122e+00, 2.51899642e+00, 2.55100334e+00,
       2.79211470e+00, 3.15770610e+00, 3.30738786e+00, 3.48786280e+00,
       3.83301435e+00, 4.71505376e+00, 5.15197888e+00, 9.57978721e+00,
       1.07508960e+01, 1.16536939e+01, 1.56223959e+01, 7.22908601e+01,
       7.27915772e+01, 8.31204299e+01, 2.51836120e+02])

In [32]:
#중복되거나 차이가 매우 작은 값 확인
for raw in unique_target_raw:
    for ratio in prices_ratios:
        if abs(float(raw) - ratio) < 0.0001:
            print(raw)
            break

0.713261649381084
1.0
1.2183406082018717
1.358422942512516


<br/>
0.713261, 1.21834, 1.35842 3가지 값이 중복되고있다.<br/>
<br/>

<br/>
이것은 제법 신기한 발견이며, 이제 나는 target 변수는 100% purchase_amount의 변화율이라고 생각한다. 하지만 이 비율이 정확히 어떤 기준으로 계산되는지는 모르겠다.<br/>
다음 card_id를 예로 들어보자.<br/>
<br/>

In [13]:
dat[dat.card_id=='C_ID_2c8d99614f'].sort_values('purchase_date')

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,purchase_amount_new,first_active_month,feature_1,feature_2,feature_3,target,target_raw
2245,Y,C_ID_2c8d99614f,-1,Y,1,B,661,M_ID_fc7d7969c3,-8,-0.7454051400,2017-06-11 23:59:54,nan,-1,8,1.0000000000,2017-06,1,2,0,0.2849175200,1.2183406082
2241,Y,C_ID_2c8d99614f,-1,Y,1,B,661,M_ID_fc7d7969c3,-8,-0.7124970800,2017-06-11 23:59:59,nan,-1,8,22.9000000000,2017-06,1,2,0,0.2849175200,1.2183406082
2250,Y,C_ID_2c8d99614f,-1,Y,1,B,661,M_ID_fc7d7969c3,-7,-0.7124970800,2017-07-11 08:50:47,nan,-1,8,22.9000000000,2017-06,1,2,0,0.2849175200,1.2183406082
2239,Y,C_ID_2c8d99614f,-1,Y,1,B,661,M_ID_fc7d7969c3,-6,-0.7124970800,2017-08-11 06:41:07,nan,-1,8,22.9000000000,2017-06,1,2,0,0.2849175200,1.2183406082
2248,Y,C_ID_2c8d99614f,-1,Y,1,B,661,M_ID_fc7d7969c3,-5,-0.7124970800,2017-09-11 18:46:07,nan,-1,8,22.9000000000,2017-06,1,2,0,0.2849175200,1.2183406082
2247,Y,C_ID_2c8d99614f,-1,Y,1,B,661,M_ID_fc7d7969c3,-4,-0.7124970800,2017-10-11 09:28:10,nan,-1,8,22.9000000000,2017-06,1,2,0,0.2849175200,1.2183406082
2246,Y,C_ID_2c8d99614f,-1,Y,1,B,661,M_ID_fc7d7969c3,-3,-0.7124970800,2017-11-11 10:46:12,nan,-1,8,22.9000000000,2017-06,1,2,0,0.2849175200,1.2183406082
2240,Y,C_ID_2c8d99614f,-1,Y,1,B,661,M_ID_fc7d7969c3,-2,-0.7124970800,2017-12-11 07:36:29,nan,-1,8,22.9000000000,2017-06,1,2,0,0.2849175200,1.2183406082
2243,N,C_ID_2c8d99614f,-1,Y,1,B,661,M_ID_fc7d7969c3,-1,-0.7124970800,2018-01-11 09:54:39,nan,-1,8,22.9000000000,2017-06,1,2,0,0.2849175200,1.2183406082
2244,N,C_ID_2c8d99614f,-1,Y,1,B,661,M_ID_fc7d7969c3,-1,-0.7124970800,2018-01-15 16:14:59,nan,-1,8,22.9000000000,2017-06,1,2,0,0.2849175200,1.2183406082


<br/>
가입 활성화 수수류 1.0을 제외해보자.<br/>
target_raw는 1.2183406082이며,이것은 27.90/22.90의 값에 해당한다.<br/>22.90은 purchase_amount_new는 22.9의 값이다.<br/>
<br/>
이제 우리는 확실히 위 card_id에서 purchase_amount는 변하였고, month_lag 1, month_lag 2에서 발생될 것을 알고있다.<br/>
이 현상은 new_mearchant_transactions 에서는 나타나지 않으며, 주최자 측에서 임의로 제외시켰다는 것을 알 수 있다. 우리가 만약 이 정보를 가지고 있다면, 우리는 쉽게 target을 예측할 수 있었을 것이다.<br/>
<br/>
따라서 정확히 target이 무엇일까.<br/>
답은 간단하다. 과거에 사용한 돈으로 사용 금액의 비율과 같다<br/>
<br/>

<br/>
데이터세트의모든 merchant 와 card에 이것이 적용되는 것 같지만,<br/>
모두 계산 하는 것은 어렵다<br/>
<br/>

# Things that are still not clear...

<br/>
우리가 미래/과거 비율을 이용하여 작업했다는 것을 알았으니,<br/>
이상값(-33.xxxx)은 실제로 사용자가 미래의 상인들에게 한푼도 쓰지 않았다는 것을 의미한다.<br/>
아쉽게도, 이것은 이 특이점들의 초기값을 데이터로 예측할 수 없음을 의미한다.<br/>
우리가 가진 데이터는 가상 데이터가 아니라, 실제 데이터임을 알 수 있다.<br/>
만약 내가 틀렸다면, Elo 기업의 가상 데이터 생성 파트가 매우 완벽히 일을 수행한 것이다.<br/>
<br/>
target이 미래/과거의 비율이라는 것을 밝혔지만, 여전히 몇 가지 의문지 남아있다.<br/>
계산에 들어가는 해당 월은 month_lag 1, month_lag 2... 등 모두 가능하다.<br/>
new_merchant-transactions가 영향을 미치는가 여부<br/>
<br/>

# Some usefule comment
<br/>
MIDfc7d7969c3 = Netflix 구독 가격 <br/>
<br/>
브라질에서! 2019년 feb당 가격은 다음과 같다: R$ 19,90 R$ 27,90 R$ 37,90<br/>
해당 사이트의 정보와 일치하는 항목: https://www.netflix.com/signup/planform<br/>
<br/>
목표값이 정말로 단지 새로운 금액/과거 금액 비율이고 ELO가 원하는 최종 모델이 단지 과거 금액을 사용해서 비율을 예측하는 것이라면, new_merchant.csv의 데이터를 사용하는 모든 모델은 결국 의미가 없다?<br/>

- 내가 이해하기로는 '그렇다'는 답이 있다. 그렇지 않으면 데이터가 유출될 것이다.
- 아니, 신제품을 더 사면 구제품도 살 확률이 높다는 의미야. 그래서 새로운 거래 데이터도 약간의 정보를 제공할 수 있다.

<br/>
